In [ ]:
import pandas as pd
import statsmodels.api as sm
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import StandardScaler
from geopy.distance import geodesic

### Create Maps from Survey Options to CoStar

In [117]:
market_map = {
    'Atlanta': 'Atlanta, GA',
    'Austin': 'Austin, TX',
    'Charlotte': 'Charlotte, NC',
    'Columbus': 'Columbus, OH',
    'DC': 'Washington, DC',
    'Dallas': 'Dallas, TX',
    'Denver': 'Denver, CO',
    'Houston': 'Houston, TX',
    'Nashville': 'Nashville, TN',
    'Orlando': 'Orlando, FL',
    'Phoenix': 'Phoenix, AZ',
    'Raleigh': 'Raleigh, NC',
    'South Florida': 'Miami, FL',
    'Tampa': 'Tampa, FL',
    'Tucson': 'Tucson, AZ'
}

manager_map = {
    'amli': 'AMLI',
    'arium': 'ARIUM',
    'avalon': 'AvalonBay',
    'bell': 'Bell',
    'bozzuto': 'Bozzuto',
    'broadstone': 'Broadstone',  
    'camden': 'Camden',
    'cortland': 'Cortland',
    'cushman_&_wakefield': 'Pinnacle', 
    'encantada': 'HSL', 
    'gables': 'Gables', 
    # 'greenwater': 'Greenwater',
    'greystar': 'Greystar',
    'hsl': 'HSL', 
    'lincoln': 'Willow Bridge', 
    'maa': 'MAA',
    'mark_taylor': 'Mark Taylor',
    'northstar': 'Northstar', 
    'northwood': 'Northwood Ravin', 
    'pb_bell': 'Bell',
    'pinnacle': 'Pinnacle',
    'post': 'Post Road',
    'rpm_living': 'RPM',
    'walton': 'Walton Communities', 
    'weidner': 'Weidner',
    'windsor': 'Windsor'
}

### Read in Raw HelloData Property Details and Reference File

In [118]:
hellodata_costar_ref = pd.read_csv('data/hellodata_costar_ref.csv')

hellodata_cols = [
    'UnitCount', 'NumberStories',
    'PropertyQuality', 'BuildingAge', 'BedroomQuality', 
    'KitchenQuality', 'BathroomQuality', 'DiningRoomQuality', 'CommonAreasQuality', 
    'FitnessCenterQuality', 'LaundryRoomQuality', 'LivingRoomQuality', 'MainEntranceQuality'
]

property_details = pd.read_csv('data/property_details.csv')
property_details = property_details[['HelloDataID'] + hellodata_cols]

property_details = pd.merge(property_details, hellodata_costar_ref, left_on='HelloDataID', right_on='property_id', how='outer').drop(columns=['property_id']).dropna(subset='HelloDataID')

C:\Users\tyson.king\AppData\Local\Temp\ipykernel_36868\2301443739.py:10: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  property_details = pd.read_csv('data/property_details.csv')


### Read in Raw CoStar Export

In [145]:
costar_export = pd.read_csv('data/branded_sites.csv')

costar_export = costar_export[['PropertyID', 'MarketName', 'SubMarketName', 'manager', 'Latitude', 'Longitude']]

costar_export = pd.merge(costar_export, property_details, left_on='PropertyID', right_on='costar_id', how='left').drop_duplicates()

### Read in Raw Survey Data and Format Columns

In [147]:
raw_survey_data = pd.read_csv('data/raw_survey_data.csv',encoding='latin1')\
    .rename(columns={'Market': 'market',
                     'Which of the following best describes your current living situation?':'living',
                     'What is your combined, annual household income?':'income',
                     'What is theÂ\xa0total monthly rent payment (for all bedrooms)Â\xa0where you live? The total rent forÂ\xa0all bedrooms, not just your portion of the rent.Â\xa0':'total_rent',
                     'What is your age?':'age',
                     'Cortland Unaided': 'cortland_unaided',
                     'Camden Unaided': 'camden_unaided',
                     'Greystar Unaided': 'greystar_unaided',
                     'MAA Unaided': 'maa_unaided'})

raw_survey_data['cortland_unaided'] = raw_survey_data['cortland_unaided'].notna().astype(int)
raw_survey_data['camden_unaided']   = raw_survey_data['camden_unaided'].notna().astype(int)
raw_survey_data['greystar_unaided'] = raw_survey_data['greystar_unaided'].notna().astype(int)
raw_survey_data['maa_unaided']      = raw_survey_data['maa_unaided'].notna().astype(int)

aided_cols = [col for col in raw_survey_data.columns if col.startswith('<strong>')]

for col in aided_cols:
    match = re.search(r'<strong>(.*?)</strong>', col)
    if match:
        brand = match.group(1).strip().lower().replace(' ', '_')
        new_col = f"{brand}_aided"
        raw_survey_data[new_col] = raw_survey_data[col].notna().astype(int)

survey_df = raw_survey_data[[
    col for col in raw_survey_data.columns
    if col in ['market', 'living', 'income', 'total_rent', 'age']
    or col.endswith('_aided') or col.endswith('_unaided')
]]

### Aggregate Survey Data to get Income, Rent and Age

In [148]:
def extract_upper_income(val):
    if 'or more' in val:
        return 200000
    match = re.search(r'under\s*\$?([\d,]+)', val)
    if match:
        return int(match.group(1).replace(',', ''))
    return None

def extract_upper_rent(val):
    if 'More than' in val:
        return 3000
    match = re.search(r'\$[\d,]+ - \$([\d,]+)', val)
    if match:
        return int(match.group(1).replace(',', ''))
    return None


survey_df['income'] = survey_df['income'].apply(extract_upper_income)
survey_df['total_rent'] = survey_df['total_rent'].apply(extract_upper_rent)

market_demos = survey_df.groupby('market').agg({
    'income': 'mean',
    'age': 'mean',
    'total_rent': 'mean'
}).reset_index()

market_demos['market'] = market_demos['market'].map(market_map)

C:\Users\tyson.king\AppData\Local\Temp\ipykernel_36868\2906079784.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df['income'] = survey_df['income'].apply(extract_upper_income)
C:\Users\tyson.king\AppData\Local\Temp\ipykernel_36868\2906079784.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df['total_rent'] = survey_df['total_rent'].apply(extract_upper_rent)


### Aggregate Survey Data to get Recognition

In [149]:
aided_cols = [col for col in raw_survey_data.columns if col.endswith('_aided')]

melted = survey_df[['market'] + aided_cols].melt(
    id_vars='market',
    value_vars=aided_cols,
    var_name='manager',
    value_name='recognized'
)

melted['manager'] = melted['manager'].str.replace('_aided', '', regex=False)

brand_recognition = (
    melted.groupby(['market', 'manager'], as_index=False)['recognized']
    .mean()
    .rename(columns={'recognized': 'recognition'})
)

brand_recognition['market'] = brand_recognition['market'].map(market_map)
brand_recognition['manager'] = brand_recognition['manager'].map(manager_map)

### Aggregate HelloData Quality Metrics

In [150]:
cols_to_exclude = ['PropertyID', 'Latitude', 'Longitude', 'HelloDataID', 'costar_id', 'SubMarketName']
cols_to_agg = [col for col in costar_export.columns if col not in cols_to_exclude + ['MarketName', 'manager']]

costar_export[cols_to_agg] = costar_export[cols_to_agg].apply(
    lambda col: col.fillna(col.mean()) if col.dtype.kind in 'biufc' else col
)

manager_metrics = (
    costar_export.groupby(['MarketName', 'manager'])[cols_to_agg]
    .mean()
    .reset_index()
)

for col in cols_to_agg:
    manager_metrics[col] = manager_metrics[col].fillna(manager_metrics[col].mean())

### Create Location Quantification Metrics

In [ ]:
costar_export['lat_bin'] = costar_export['Latitude'].round(2)
costar_export['lon_bin'] = costar_export['Longitude'].round(2)

tile_density = (
    costar_export.groupby(['MarketName', 'lat_bin', 'lon_bin'])
    .agg(
        total_units=('UnitCount', 'sum'),
        total_assets=('PropertyID', 'count')
    )
    .reset_index()
)

top_tiles = (
    tile_density.sort_values(['MarketName', 'total_units'], ascending=[True, False])
    .groupby('MarketName')
    .head(3)  # Adjust to 2, 5, etc. depending on your needs
    .copy()
)

In [161]:
top_tiles[top_tiles['MarketName'] == 'Boston, MA']

,MarketName,lat_bin,lon_bin,total_units,total_assets,center_lat,center_lon
5443,"Boston, MA",42.34,-71.08,6759.454800,27,42.34,-71.08
5441,"Boston, MA",42.34,-71.10,6008.404267,24,42.34,-71.10
5461,"Boston, MA",42.35,-71.14,4756.653378,19,42.35,-71.14


### Final Aggregation

In [127]:
final_metrics = pd.merge(manager_metrics, brand_recognition, left_on=['MarketName', 'manager'], right_on=['market', 'manager'], how='right')
final_metrics = pd.merge(final_metrics, market_demos, on='market', how='left')

final_metrics = final_metrics.drop(columns=['MarketName']).dropna(subset='NumberStories')

### Train Model

In [128]:
x = final_metrics.drop(columns=['market', 'manager', 'recognition'])
y = final_metrics['recognition']

model = sm.OLS(y, x)
results = model.fit()

print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:            recognition   R-squared (uncentered):                   0.717
Model:                            OLS   Adj. R-squared (uncentered):              0.689
Method:                 Least Squares   F-statistic:                              25.83
Date:                Wed, 09 Jul 2025   Prob (F-statistic):                    1.33e-36
Time:                        10:23:20   Log-Likelihood:                          141.85
No. Observations:                 179   AIC:                                     -251.7
Df Residuals:                     163   BIC:                                     -200.7
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [ ]:
# Prepare data
X = final_metrics.drop(columns=['market', 'manager', 'recognition'])
y = final_metrics['recognition']

# Standardize predictors
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# ElasticNetCV for feature selection
enet = ElasticNetCV(cv=5, l1_ratio=0.5, random_state=42)
enet.fit(X_scaled, y)

# Get selected features (non-zero coefficients)
selected_features = X.columns[enet.coef_ != 0]

# Refit OLS on selected features
X_selected = sm.add_constant(X[selected_features])  # add intercept
ols_model = sm.OLS(y, X_selected)
ols_results = ols_model.fit()

# Print results
print(ols_results.summary())

### Diagnostic Plots

In [ ]:
# Get fitted values and residuals
fitted_vals = results.fittedvalues
residuals = results.resid
standardized_residuals = results.get_influence().resid_studentized_internal
leverage = results.get_influence().hat_matrix_diag

# 1. Residuals vs Fitted
plt.figure(figsize=(6, 4))
sns.residplot(x=fitted_vals, y=y, lowess=True, line_kws={'color': 'red'})
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted')
plt.axhline(0, linestyle='--', color='gray')
plt.tight_layout()
plt.show()

# 2. Normal Q-Q
sm.qqplot(standardized_residuals, line='45', fit=True)
plt.title('Normal Q-Q')
plt.tight_layout()
plt.show()

# 3. Scale-Location (Spread vs Fitted)
plt.figure(figsize=(6, 4))
plt.scatter(fitted_vals, abs(standardized_residuals) ** 0.5)
sns.regplot(x=fitted_vals, y=abs(standardized_residuals) ** 0.5,
            scatter=False, lowess=True, line_kws={'color': 'red'})
plt.xlabel('Fitted values')
plt.ylabel('√|Standardized Residuals|')
plt.title('Scale-Location')
plt.tight_layout()
plt.show()

# 4. Leverage vs. Standardized Residuals
plt.figure(figsize=(6, 4))
plt.scatter(leverage, standardized_residuals)
sns.regplot(x=leverage, y=standardized_residuals, scatter=False,
            lowess=True, line_kws={'color': 'red'})
plt.xlabel('Leverage')
plt.ylabel('Standardized Residuals')
plt.title('Residuals vs Leverage')
plt.axhline(0, linestyle='--', color='gray')
plt.tight_layout()
plt.show()